In [1]:
import os
import re
import kss
from tqdm.notebook import tqdm
from konlpy.tag import Mecab
mecab = Mecab()

In [3]:
path = 'text/'
folder_list = os.listdir(path)

file_list = []
for folder in folder_list:
    folder_path = path + folder + '/'
    txt_list = os.listdir(folder_path)
    for txt in txt_list:
        file_list.append(folder_path + txt)
        
with open('kowiki_210420.txt', 'w', encoding='utf-8') as full:
    kowiki = []
    for file in tqdm(file_list):
        with open(file, 'r', encoding='utf-8') as t:
            content = t.read()
            kowiki.append(content)
    cont = ''.join(kowiki)
    full.write(cont)
    
docs_sp = re.split('<doc[^>]+>', cont)

doc_ob = []
doc_ex = []
for idx, doc in enumerate(docs_sp):
    doc_arr = []
    try:
        doc_arr = doc.split('\n\n')
        title = doc_arr[0].replace('\n', '')
        ob = {'title' : title}
    
        content = doc_arr[1]
        ob['content'] = content
        
        doc_ob.append(ob)
    except:
        doc_ex.append(doc)
        
del cont
del kowiki
del docs_sp
        
filter_arr = []
for doc in tqdm(doc_ob):
    filter_arr.append(doc['title'])
    filter_arr.append('\n')
    filter_arr.append(doc['content'])
    filter_arr.append('\n\n')
last_text = ''.join(filter_arr)

del doc_ob
del doc_ex
del filter_arr

docs = last_text.split('\n\n')
docs_split = []
for doc in tqdm(docs):
    aa = re.sub('\(\)|\(,?\s?\)', '', doc)  # (, ) 제거
    bb = re.sub('\(([^)]+)\)', '', aa)  # () 괄호와 괄호 안 내용 제거
    cc = re.sub('\n[\w|\s]{2,20}.\n', '\n', bb)  # 소제목 (글자 + . 로 구성된 행) 제거
    
    for_title_arr = cc.split('\n')
    title = for_title_arr[0]
    content = cc[len(title) + 1:]
    content_s = content + '\n\n'
    docs_split.append(content_s)
docs_split_join = ''.join(docs_split)

del docs

wiki_docs = docs_split_join.split('\n\n')

wiki_docs_len = []
for idx, doc in enumerate(wiki_docs):
    doc_len = len(doc)
    len_ob = {'doc_id':idx, 'doc_len':doc_len, 'content':doc}
    wiki_docs_len.append(len_ob)
    
del docs_split_join
del wiki_docs
    
wiki_docs_long = []
one_sent_docs = []
for doc in tqdm(wiki_docs_len):
    content = doc['content']
    if len(kss.split_sentences(content)) == 1:
        one_sent_docs.append(doc)
    else:
        wiki_docs_long.append(doc)
        
del wiki_docs_len
        
long_arr = []
for doc in wiki_docs_long:
    long_arr.append(doc['content'])
    long_arr.append('\n\n')
    
del wiki_docs_long
    
wiki_n = ''.join(long_arr)
useless = wiki_n.split('\n\n')
useless_1 = [doc.replace('&lt;', '').replace('&gt;', '').replace('&nbsp;', '') for doc in useless]
useless_2 = [doc.replace('&amp;', '').replace('&quot;', '').replace('&#039;', '') for doc in useless_1]
useless_3 = [re.sub('\(([^)]+)\)', '', doc) for doc in useless_2]
useless_4 = list(filter(lambda x: 'formula' not in x, useless_3))

del long_arr
del wiki_n

with open('kowiki_210420_r.txt', 'w', encoding='utf-8') as f:
    txt = '\n\n'.join(useless_4)
    f.write(txt)

  0%|          | 0/787 [00:00<?, ?it/s]

  0%|          | 0/1198709 [00:00<?, ?it/s]

  0%|          | 0/1865968 [00:00<?, ?it/s]

  0%|          | 0/1865992 [00:00<?, ?it/s]

In [4]:
with open('kowiki_210420_r.txt', 'r', encoding='utf-8') as f:
    read_lines = f.readlines()
    
len_smaller_2 = list(filter(lambda x: len(x) < 3 and x != '\n', read_lines))

fixed_lines = []
for line in read_lines:
    if len(line) < 3 and line != '\n':
        line = re.sub('.\\n', '\n', line)
    fixed_lines.append(line)
    
del read_lines
    
long_lines = list(filter(lambda x: len(x) > 15 or x == '\n' in x, fixed_lines))

del fixed_lines

re_splited = []
sents_ss = []
for line in tqdm(long_lines):
    if line == '\n':
        re_splited.append(line)
    else:
        sents_ss = kss.split_sentences(line)
        for sent in sents_ss:
            re_splited.append(sent + '\n')
    sents_ss = []
    
del long_lines
    
line_morphs = []
for line in tqdm(re_splited):
    morphs_arr = mecab.morphs(line)
    morphs_sent = ' '.join(morphs_arr)
    line_morphs.append(morphs_sent)
    
del re_splited
    
final_arr = []
for line in tqdm(line_morphs):
    if len(line) == 0:
        line_with_s = '\n'
    else:
        line_with_s = line + '\n'
    final_arr.append(line_with_s)
    
del line_morphs
    
final_text = ''.join(final_arr)
with open('kowiki_210401_final.txt', 'w', encoding='utf-8') as f:
    f.write(final_text)
    
del final_text
del final_arr

  0%|          | 0/3845931 [00:00<?, ?it/s]

  0%|          | 0/6477594 [00:00<?, ?it/s]

  0%|          | 0/6477594 [00:00<?, ?it/s]